#Install Stage

Clone Stable diffusion webui git repo

In [ ]:
#@markdown Setup webUI
instant_replace = False #@param {type: "boolean"}
import os
if instant_replace and os.path.exists('/content/stable-diffusion-webui'):
  !sudo rm -r /content/stable-diffusion-webui

!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git

Extension Installer

In [ ]:
!!apt-get -y install -qq aria2
!!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/Kimyobu/Stable-diffusion-webui-colab/raw/main/extensions.txt
print(open('/content/extensions.txt').read())#@markdown Check list of default common extension<br>ดู extension ทั่วไปที่จัดไว้ให้

In [ ]:
!apt-get -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/Kimyobu/Stable-diffusion-webui-colab/raw/main/extensions.txt
#@markdown Leave it default to install common extensions<br>ไม่ต้องเปลี่ยนถ้าจะลง extension ทั่วไป
extension_list_file = '/content/extensions.txt' #@param {type: "string"}
install = open(extension_list_file, 'r').read().split('\n')
%cd /content/stable-diffusion-webui/extensions
for x in install :
  !git clone {x}

#Model import

In [ ]:
#@markdown Import all saved models from google drive saved folder
from google.colab import drive
drive.mount('/content/drive')
import os
save_path =  '/content/drive/MyDrive/kimyobu-colab/models/'
model_path = '/content/stable-diffusion-webui/models/'
for x in os.listdir(save_path):
   for f in open(save_path + x).read().split('\n'):
      fol = model_path + x.split('.')[0]
      !mkdir -p {fol}
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {f} -d {fol}

Model downloader<br>
Put your model link file and select model type<br>
ใส่ลิงค์ไฟล์โมเดลแล้วเลือกประเภทโมเดล

In [ ]:
#@markdown Install to Runtime storage<br>อันนี้มันจะลงที่ Runtime
!apt-get -y install -qq aria2
import os
save_path = '/content/stable-diffusion-webui/models/'
model_type = 'Stable-diffusion' #@param ["adetailer","deepdanbooru","karlo","sam","VAE-approx","Codeformer","ESRGAN","LDSR","Stable-diffusion","ControlNet","GFPGAN","Lora","SwinIR","deepbooru","hypernetworks","LyCORIS","VAE"]
model_path = save_path + model_type
model_url = 'https://civitai.com/api/download/models/77276' #@param {type:"string"}
model_url = f"\"{model_url}\""
!mkdir -p "{model_path}"

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d {model_path}

In [ ]:
#@markdown Install to Google_Drive storage<br>อันนี้มันจะลงที่ Google_Drive
from google.colab import drive
drive.mount('/content/drive')

!apt-get -y install -qq aria2
model_type = 'Lora' #@param ["adetailer","deepdanbooru","karlo","sam","VAE-approx","Codeformer","ESRGAN","LDSR","Stable-diffusion","ControlNet","GFPGAN","Lora","SwinIR","deepbooru","hypernetworks","LyCORIS","VAE"]
model_url = 'https://civitai.com/api/download/models/107366' #@param {type:"string"}
model_path = '/content/stable-diffusion-webui/models/' + model_type
save_path =  '/content/drive/MyDrive/kimyobu-colab/models/'
!mkdir -p {save_path}
open(save_path + model_type + '.txt', 'a').write(model_url + "\n")
model_url = f"\"{model_url}\""

#Import state
import_to_runtime = True #@param {type: "boolean"}
if import_to_runtime:
  !mkdir -p {model_path}
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d {model_path}

#Launch WebUI

In [ ]:
%cd /content/stable-diffusion-webui
!pip install --upgrade fastapi
#!git stash
#!git pull origin master
#!git checkout b6af0a3809ea869fb180633f9affcae4b199ffcf
#!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117
#!pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
#!pip install -q xformers==0.0.18 triton==2.0.0 -U
!apt install libcairo2-dev pkg-config python3-dev
!pip install svglib
!pip install pytorch-lightning==1.6.5
!pip install torchmetrics==0.11.0
#!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' /content/alohomora/modules/shared.py
# Web UI tunnel
!COMMANDLINE_ARGS="--share --no-half-vae --xformers --enable-insecure-extension- --theme dark --gradio-queue" REQS_FILE="requirements.txt" python launch.py

In [ ]:
#@markdown Save generated files
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/stable-diffusion-webui/outputs/* /content/drive/MyDrive/kimyobu-colab/outputs